In [1]:
# Generate source data for various state from the national dataset.
# for Spark application configuration such as executor memory setting, see this:
# https://docs.cloudera.com/runtime/7.1.0/running-spark-applications/topics/spark-configure-apps.html
# https://docs.cloudera.com/runtime/7.1.0/running-spark-applications/spark-running-applications.pdf
# https://docs.cloudera.com/runtime/7.1.0/howto-data-science.html

import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import operator

In [2]:
IPUMS_FILE = '/data/input/EXT1940USCB.dat'

In [3]:
spark = SparkSession.builder \
    .appName("Jay")          \
    .master("spark://das11.mitre.org:7077")    \
    .config("spark.executor.memory", "24g")    \
    .config("spark.executor.instances", "3")    \
    .config("spark.executor.cores", "8")    \
    .getOrCreate()
spark

In [ ]:
#spark_session.conf.set("spark.executor.memory", '8g')
#spark_session.conf.set('spark.executor.cores', '3')
#spark_session.conf.set('spark.cores.max', '3')
#spark_session.conf.set("spark.driver.memory",'8g')
#spark.conf.set("spark.executor.instances", 3)
#spark.conf.set("spark.executor.cores", 4)

In [4]:
rdd = spark.sparkContext.textFile(IPUMS_FILE)
rdd.take(10)

['H19400200000001022600000001000002700000000001198642810230001100000009999000000100000990000840199990011100000000200002000011000100107000000001000200000000100020',
 'P19400200000001000100000001000000000000100101000000000000000009999010101207399500000110000000530530000000000000000000000000010114140050501102071275201198361125260009999999999990000000907302210077016402700009990000999999979999990000000F92A1965-DD8D-4DA5-9EDA-4DEF7D1F338C010',
 'P1940020000000100020000000100000000000010020100000000000000000999912120310699960000011000000400400000000000000000000040000001011212003030110209889700001105220000000999999999999099999812008175039700300251000999000099999997999999000000069FA0B5D-4C58-4AD5-BDAC-0CA2EE215777020',
 'H19400200000002047600000001000004300000000001198642810230001100000009999000000100000990000840199990011100000000500003001022200100107000000002000400000000200040',
 'P19400200000002000100000001000000000000100102000020100000000009999010101203599200000110000000190190001901900000000

In [5]:
WY_UNIT_RDD = rdd.filter(lambda x: "56" in x[53:55])
WY_UNIT_RDD.take(10)

['H19400238412818036000000001000003100000000001198641685600101100000009999000000100000500000840199990012200020999999901011122100100102000000001000300000000100030',
 'H19400238412819022800000001000002100000000011198641685600101100000009999000000100000500000840199990012200015999999901010011000100101000000002000200000000200020',
 'H19400238412820018800000001000001600000000001198641685600101100000009999000000100000500000840199990012200010999999901000011000100101000000003000100000000300010',
 'H19400238412821049600000001000004100000000001198641685600101100000009999000000100000500000840199990012200030999999901011122100100101000000004000400000000400040',
 'H19400238412822024900000001000002100000000001198641685600101100000009999000000100000500000840199990012200030999999901010011000100101000000005000200000000500020',
 'H19400238412823029000000001000002100000000001198641685600101100000009999000000100000500000840199990012200015999999901010011000100101000000006000200000000600020',
 'H1940023841282

In [15]:
WY_UNIT_RDD.count()

80146

In [6]:
WY_SERIAL_RDD = WY_UNIT_RDD.map(lambda x: x[7:15])
WY_SERIAL_RDD.take(10)

['38412818',
 '38412819',
 '38412820',
 '38412821',
 '38412822',
 '38412823',
 '38412824',
 '38412825',
 '38412826',
 '38412827']

In [20]:
WY_SERIAL_RDD.count()

80146

In [7]:
serial_list = WY_SERIAL_RDD.collect()
serial_list[:10]

['38412818',
 '38412819',
 '38412820',
 '38412821',
 '38412822',
 '38412823',
 '38412824',
 '38412825',
 '38412826',
 '38412827']

In [ ]:
# This is taking forever, after 2 hours and still running.
# So I stopped it and switched to pyspark dataframe using semi outer join.

WY_RDD = rdd.filter(lambda x: x[7:15] in serial_list)
WY_RDD.take(10)

In [ ]:
WY_RDD.count()

In [ ]:
WY_RDD.saveAsTextFile("file:///data/input/Wyoming.dat")